<a href="https://colab.research.google.com/github/bahador1/I-GOS/blob/main/dlg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- our little squirrel must be
    - `torch.float32`


- speculations:
    - the picture of dog is more complicated than the picutre of tree!!! like manifold and shit  
---

- task:
    - simplifying the reading data section

In [63]:
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import pickle
import PIL.Image as Image


class LeNet(nn.Module):
    def __init__(self, channel=3, hideen=768, num_classes=10):
        super(LeNet, self).__init__()
        act = nn.Sigmoid
        self.body = nn.Sequential(
            nn.Conv2d(channel, 12, kernel_size=5, padding=5 // 2, stride=2),
            act(),
            nn.Conv2d(12, 12, kernel_size=5, padding=5 // 2, stride=2),
            act(),
            nn.Conv2d(12, 12, kernel_size=5, padding=5 // 2, stride=1),
            act(),
        )
        self.fc = nn.Sequential(
            nn.Linear(hideen, num_classes)
        )

    def forward(self, x):
        out = self.body(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def weights_init(m):
    try:
        if hasattr(m, "weight"):
            m.weight.data.uniform_(-0.5, 0.5)
    except Exception:
        print('warning: failed in weights_init for %s.weight' % m._get_name())
    try:
        if hasattr(m, "bias"):
            m.bias.data.uniform_(-0.5, 0.5)
    except Exception:
        print('warning: failed in weights_init for %s.bias' % m._get_name())


In [64]:
class Dataset_from_Image(Dataset):
    def __init__(self, imgs, labs, transform=None):
        self.imgs = imgs # img paths
        self.labs = labs # labs is ndarray
        self.transform = transform
        del imgs, labs

    def __len__(self):
        return self.labs.shape[0]

    def __getitem__(self, idx):
        lab = self.labs[idx]
        img = Image.open(self.imgs[idx])
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img = self.transform(img)
        return img, lab


In [65]:
dataset = 'MNIST'
root_path = '.'
data_path = os.path.join(root_path, './data').replace('\\', '/')
save_path = os.path.join(root_path, 'results/DLG_%s'%dataset).replace('\\', '/')

lr = 1.0
num_dummy = 1
Iteration = 300
num_exp = 1

use_cuda = torch.cuda.is_available()
device = 'cuda' if use_cuda else 'cpu'

# tt = transforms.Compose([transforms.ToTensor()])
# tp = transforms.Compose([transforms.ToPILImage()])
to_tensor = transforms.ToTensor()
to_pil = transforms.ToPILImage()

print(dataset, 'root_path:', root_path)
print(dataset, 'data_path:', data_path)
print(dataset, 'save_path:', save_path)

if not os.path.exists('results'):
    os.mkdir('results')
if not os.path.exists(save_path):
    os.mkdir(save_path)



''' load data '''
if dataset == 'MNIST':
    shape_img = (28, 28)
    num_classes = 10
    channel = 1
    hidden = 588
    dst = datasets.MNIST(data_path, download=True)

elif dataset == 'cifar100':
    shape_img = (32, 32)
    num_classes = 100
    channel = 3
    hidden = 768
    dst = datasets.CIFAR100(data_path, download=False)

MNIST root_path: .
MNIST data_path: ././data
MNIST save_path: ./results/DLG_MNIST


In [66]:
data_idx  = 0
tmp_datum = to_tensor(dst[data_idx][0]).to(device) # the image
gt_data = tmp_datum.unsqueeze(0)

tmp_label = torch.Tensor([dst[data_idx][1]]).long().to(device) # the label
gt_label = tmp_label.view(1, )


In [67]:


''' train DLG '''
for idx_net in range(num_exp:=1):
    net = LeNet(channel=channel, hideen=hidden, num_classes=num_classes)
    net.apply(weights_init)

    print('running %d|%d experiment'%(idx_net, num_exp))
    net = net.to(device)
    idx_shuffle = np.random.permutation(len(dst))


    criterion = nn.CrossEntropyLoss().to(device)
    imidx_list = []

    # compute original gradient
    out = net(gt_data)
    y = criterion(out, gt_label)
    dy_dx = torch.autograd.grad(y, net.parameters())
    original_dy_dx = list((_.detach().clone() for _ in dy_dx))

    # generate dummy data and label
    dummy_data = torch.randn(gt_data.size()).to(device).requires_grad_(True)
    dummy_label = torch.randn((gt_data.shape[0], num_classes)).to(device).requires_grad_(True)

    optimizer = torch.optim.LBFGS([dummy_data, dummy_label], lr=lr)


    history = []
    history_iters = []
    losses = []
    mses = []
    train_iters = []

    print('lr =', lr)
    for iters in range(Iteration):

        def closure():
            optimizer.zero_grad()
            pred = net(dummy_data)
            dummy_loss = - torch.mean(torch.sum(torch.softmax(dummy_label, -1) * torch.log(torch.softmax(pred, -1)), dim=-1))
            # dummy_loss = criterion(pred, gt_label)


            dummy_dy_dx = torch.autograd.grad(dummy_loss, net.parameters(), create_graph=True)

            grad_diff = 0
            for gx, gy in zip(dummy_dy_dx, original_dy_dx):
                grad_diff += ((gx - gy) ** 2).sum()
            grad_diff.backward()
            return grad_diff

        optimizer.step(closure)
        current_loss = closure().item()
        train_iters.append(iters)
        losses.append(current_loss)
        mses.append(torch.mean((dummy_data-gt_data)**2).item())


        if iters % int(Iteration / 30) == 0:
            current_time = str(time.strftime("[%Y-%m-%d %H:%M:%S]", time.localtime()))
            print(current_time, iters, 'loss = %.8f, mse = %.8f' %(current_loss, mses[-1]))
            history.append([to_pil(dummy_data[imidx].cpu()) for imidx in range(num_dummy)])
            history_iters.append(iters)

            for imidx in range(num_dummy):
                plt.figure(figsize=(12, 8))
                plt.subplot(3, 10, 1)
                plt.imshow(to_pil(gt_data[imidx].cpu()))
                for i in range(min(len(history), 29)):
                    plt.subplot(3, 10, i + 2)
                    plt.imshow(history[i][imidx])
                    plt.title('iter=%d' % (history_iters[i]))
                    plt.axis('off')

                plt.savefig('%s/DLG_on_%s.png' % (save_path,data_idx))
                plt.close()


            if current_loss < 0.000001: # converge
                break

        loss_DLG = losses
        label_DLG = torch.argmax(dummy_label, dim=-1).detach().item()
        mse_DLG = mses


    print('imidx_list:', imidx_list)
    print('loss_DLG:', loss_DLG[-1],)
    print('mse_DLG:', mse_DLG[-1],)
    print('gt_label:', gt_label.detach().cpu().data.numpy(), 'lab_DLG:', label_DLG)

    print('----------------------\n\n')

running 0|1 experiment
lr = 1.0
[2025-09-16 09:26:13] 0 loss = 4.85508108, mse = 0.79746544
[2025-09-16 09:26:15] 10 loss = 0.00020117, mse = 0.00018950
[2025-09-16 09:26:17] 20 loss = 0.00001581, mse = 0.00001245
[2025-09-16 09:26:20] 30 loss = 0.00000033, mse = 0.00000056
imidx_list: []
loss_DLG: 3.3059203019547567e-07
mse_DLG: 5.556934752348752e-07
gt_label: [5] lab_DLG: 5
----------------------


